In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6,6))

# Parameters for multiplicative clamped rate
r = 0.88            # constant multiplier (decay factor < 1)
steps = 11          # number of "rewards"
y0 = 0.9            # starting value
y_min, y_max = 0.0, 1.0  # clamps

# Generate clamped multiplicative curve
y_clamped = [y0]
for _ in range(1, steps):
    next_val = y_clamped[-1] * r
    next_val = min(y_max, max(y_min, next_val))
    y_clamped.append(next_val)

x = np.arange(steps)

# Plot your exponential for comparison
a = 0.9
c = 0.1284
d = 0
y_exp = a * np.exp(-c * x) + d

ax.plot(x, y_exp, color="blue", marker="o", label="Exponential decay")
ax.plot(x, y_clamped, color="red", marker="s", label="Clamped multiplicative rate")

ax.set_xlabel('Rewards collected')
ax.set_ylabel('P(reward)')
ax.set_ylim(-0.1,1)
ax.set_xlim(-0.5,10.5)
ax.legend()
plt.show()


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,6))
marker = 'o'
ax1 = ax

x = np.linspace(0, 9, 10)  # Generate 100 points between 0 and 5
a = 0.6
b = math.e  # Amplitude
c = 0.1284
d = 0

# Generate x values
y = a * pow(b, -c * x) + d

ax1.plot(x, y, color=color2, marker=marker)
# ax1.text(0.15, 0.15, f'a = {a}', color=color2)

a = 0.9
y = a * pow(b, -c * x) + d
ax1.plot(x, y, color=color1, marker=marker)
# ax1.text(2, 0.75, f'a = {a}', color=color1)

a = 0.0
y = a * pow(b, -c * x) + d
ax1.plot(x, y, color=color3, marker=marker)
# ax1.text(3, 0.05, f'a = {a}', color=color3)

ax1.set_xlabel('Rewards collected')
ax1.set_ylabel('P(reward)')
# plt.hlines(0.6, 10, 0.6, color='k', linestyle='--')
ax1.set_ylim(-0.1,1)
ax1.set_xlim(-0.5,10.5)

sns.despine()
plt.tight_layout()
plot  = all_epochs.groupby(['patch_label', 'cumulative_rewards'])['reward_probability'].first().reset_index()
sns.lineplot(data=plot, x='cumulative_rewards', y='reward_probability', hue='patch_label', ax=ax1, marker='X', palette=color_dict_label)
fig.savefig(foraging_figures+'\schematic task.svg', dpi=300)